In [1]:
import openai
import llm2geneset
import pandas as pd
import random

aclient = openai.AsyncClient()

In [2]:
al002 = pd.read_csv("data/Stats.TECAN.AL002_vs_isotype.txt", sep="\t")
al002 = al002[al002["PercentDetectedSamples"] > 0.9]

In [3]:
bgd_genes = al002["gene"].to_list()
al002_sig = al002[(al002["padj"] < 0.1) ]
genes = al002_sig["gene"].to_list()

In [4]:
random.seed(138337)
random.shuffle(genes)

In [5]:
print("\n".join(genes))

LILRB2
MX1
GSN
MFGE8
CCL3
GPD1
TRPV2
DHCR24
TNFAIP3
MATK
CXCL8
CHIT1
TPRKB
RASSF4
SMC6
FOS
RHOBTB2
RNF19B
FBP1
SLC11A1
DCSTAMP
HSD3B7
OAS2
TGM2
MCEMP1
FAM20C
ACP5
CD9
AP5B1


In [45]:
context_str = ""
llm2geneset_no_context_res = await llm2geneset.gs_proposal_bench(aclient, [genes], model="gpt-4o-2024-05-13", context=context_str,
                                                                 bgd_genes=bgd_genes,
                                                                 n_pathways=50)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:17<00:00, 17.90s/it]


In [47]:
df = llm2geneset_no_context_res[0]["ora_results"]
#df.to_csv("outputs/no_context_AL002.tsv",sep="\t",index=None)
df[df["p_adj"] < 0.01]

,set_descr,generatio,bgratio,richFactor,foldEnrich,p_val,p_adj,intersection,set_genes,ngenes,nset,ninter,in_toks,out_toks
13,Osteoclast differentiation,0.103448,0.002818,0.100000,36.713793,0.000070,0.003510,"DCSTAMP,FOS,ACP5","MMP9,TRAF6,IL1B,IL6,MAPK14,VAV3,ITGB3,PIK3CA,R...",29,30,3,158,434
21,Gluconeogenesis,0.068966,0.001033,0.181818,66.752351,0.000388,0.009701,"GPD1,FBP1","PCK2,GPD1,PDP1,PFKL,GYS1,FBP1,PC,GPD2,PFKP,PFK...",29,11,2,156,208


In [42]:
context_str = "in vitro microglia treated with a TREM2 agonist antibody"
llm2geneset_context_res = await llm2geneset.gs_proposal_bench(aclient, [genes], model="gpt-4o-2024-05-13", 
                                                              context=context_str,
                                                              bgd_genes=bgd_genes,
                                                              n_pathways=50)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:20<00:00, 20.71s/it]


In [44]:
df = llm2geneset_context_res[0]["ora_results"]
#df.to_csv("outputs/with_context_AL002.tsv",sep="\t",index=None)
df[df["p_adj"] < 0.01]

,set_descr,generatio,bgratio,richFactor,foldEnrich,p_val,p_adj,intersection,set_genes,ngenes,nset,ninter,in_toks,out_toks
1,Regulation of innate immune response,0.137931,0.005166,0.072727,26.70094,0.000014,0.000701,"CXCL8,TNFAIP3,OAS2,MX1","STAT2,TRAF6,OAS3,CCL2,CLEC7A,NLRP3,TGFB1,RELA,...",29,55,4,160,618


In [31]:
gmt = llm2geneset.read_gmt("libs_human/gmt/KEGG_2021_Human.txt")
set_descr = gmt["descr"]
gene_sets = gmt["genes"]
df = llm2geneset.simple_ora(genes, set_descr, gene_sets, bgd_genes=bgd_genes)
#df.to_csv("outputs/KEGG_AL002.tsv",sep="\t",index=None)
df[df["p_adj"] < 0.05]

,set_descr,oddr,generatio,bgratio,richFactor,foldEnrich,p_val,p_adj,intersection,set_genes,ngenes,nset,ninter
251,Rheumatoid arthritis,30.294118,0.137931,0.006105,0.061538,22.593103,0.000027,0.008574,"CXCL8,FOS,ACP5,CCL3","TCIRG1,ATP6V0E1,ATP6V1A,TGFB2,CCL3,CCL2,HLA-DM...",29,65,4
161,Measles,19.242609,0.137931,0.009392,0.040000,14.685517,0.000145,0.023246,"FOS,TNFAIP3,OAS2,MX1","PIK3R3,TP53,TYK2,FCGR2B,IL1B,MYD88,BAD,HSPA8,S...",29,100,4
173,NOD-like receptor signaling pathway,14.521107,0.137931,0.012304,0.030534,11.210319,0.000410,0.042650,"CXCL8,TNFAIP3,TRPV2,OAS2","NOD1,CCL2,TRAF2,DHX33,TYK2,ATG12,IL1B,MYD88,TX...",29,131,4
136,IL-17 signaling pathway,24.691100,0.103448,0.005541,0.050847,18.668030,0.000533,0.042650,"CXCL8,FOS,TNFAIP3","PTGS2,CXCL3,MMP9,TRAF6,SRSF1,CCL2,GSK3B,TRAF2,...",29,59,3
64,Coronavirus disease,11.499175,0.137931,0.015403,0.024390,8.954584,0.000954,0.045449,"CXCL8,FOS,OAS2,MX1","CGAS,PIK3R3,RPS16,PLCG1,CCL2,RPL27A,TYK2,IL1B,...",29,164,4
290,Toll-like receptor signaling pathway,19.761675,0.103448,0.006856,0.041096,15.087860,0.000994,0.045449,"CXCL8,FOS,CCL3","PIK3R3,TOLLIP,TRAF6,TIRAP,CCL3,MAP2K3,RELA,IL1...",29,73,3
50,Chagas disease,19.761675,0.103448,0.006856,0.041096,15.087860,0.000994,0.045449,"CXCL8,FOS,CCL3","PIK3R3,TRAF6,TGFB2,CCL3,CCL2,TGFB1,RELA,C1QA,I...",29,73,3
